In [41]:
import cv2
import glob
import matplotlib.pyplot as plt
import numpy as np
import os.path
import pickle
%matplotlib inline

def threshold_image(img):
  height, width, _ = img.shape

  # RGB color mask
  rgb_low = np.array([200, 200, 20])
  rgb_high = np.array([255]*3)
  rgb_binary = cv2.inRange(img, rgb_low, rgb_high).astype(np.bool)
  
  # S channel mask
  hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
  s_channel = hls[:,:, 2]
  s_channel_binary = np.zeros_like(s_channel)
  s_channel_binary[s_channel >= 150] = 1
  
  # Gradient mask
  gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
  kernel_size = 5
  blur_gray = cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)
  edges = cv2.Canny(blur_gray, 50, 150).astype(np.bool)
  
  combined_color = np.zeros_like(img)
  combined_color[rgb_binary, 0] = 255
  combined_color[(s_channel_binary > 0), 1] = 255
  combined_color[edges, 2] = 255

  combined_binary = np.zeros_like(gray)
  combined_binary[rgb_binary | (s_channel_binary > 0) | edges] = 255
  return combined_color, combined_binary
  

In [ ]:
images = glob.glob("test_images/*.jpg")
fig, axes = plt.subplots(len(images), 3, figsize=(30, 10*len(images)))
fig.tight_layout()
for i, fname in enumerate(images):
  img = cv2.imread(fname)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  combined_color, combined_binary = threshold_image(img)
  base_name = os.path.basename(fname)
  cv2.imwrite("output_images/" + base_name, combined_binary)
  axes[i, 0].imshow(img)
  axes[i, 0].set_title(base_name, fontsize=32)
  axes[i, 1].imshow(combined_color)
  axes[i, 2].imshow(combined_binary, cmap = 'gray')